In [23]:
import pandas as pd

# 加载Excel文件
xls = pd.ExcelFile("E:\\数学建模国赛2023选题\\C题\\第二题建模约束参数.xlsx")


In [24]:
# 查看Excel文件中各个sheet的前几行数据以了解结构和列名
sheet_names = xls.sheet_names

# 存储每个sheet的前几行数据
sheet_previews = {}

for sheet in sheet_names:
    sheet_previews[sheet] = pd.read_excel(xls, sheet).head()

sheet_previews


{'Model_Params':            a         b   星期   分类名称
 0  84.277492  0.974480  星期三  水生根茎类
 1  81.092421  1.009424  星期三    花叶类
 2  14.822394  1.011291  星期三    花菜类
 3  25.241518  0.999187  星期三     茄类
 4  61.474034  1.000100  星期三    辣椒类,
 'Markup_Bounds':     分类名称  lower_bound  upper_bound
 0  水生根茎类    44.059830    96.064736
 1    花叶类    55.054544    97.460568
 2    花菜类    36.620075    83.754887
 3     茄类    43.469963   134.360511
 4    辣椒类    46.272717   148.951008,
 'Sales_Bounds':     分类名称  lower_bound  upper_bound
 0  水生根茎类      4.82050     91.51175
 1    花叶类    105.64975    289.32900
 2    花菜类      8.09800     79.35975
 3     茄类     12.86075     51.80800
 4    辣椒类     33.14950    151.92225,
 'Daily_Sales_Bounds':    lower_bound  upper_bound
 0    228.75775    650.33475}

In [25]:
model_params = pd.read_excel(xls, 'Model_Params')

# 读取成本加价范围约束
markup_bounds = pd.read_excel(xls, 'Markup_Bounds')

# 读取销量范围约束
sales_bounds = pd.read_excel(xls, 'Sales_Bounds')

# 读取每日销售总量约束
daily_sales_bounds = pd.read_excel(xls, 'Daily_Sales_Bounds')

In [26]:
!pip install pyswarm

In [28]:
import numpy as np
from pyswarm import pso

# 定义目标函数
def objective(X):
    x = X[:42]
    y = X[42:]
    return -np.sum(x * y)

# 从Excel读取的模型参数和约束条件
a_values = model_params['a'].values
b_values = model_params['b'].values

markup_lower_bound = markup_bounds['lower_bound'].values
markup_upper_bound = markup_bounds['upper_bound'].values

sales_lower_bound = sales_bounds['lower_bound'].values
sales_upper_bound = sales_bounds['upper_bound'].values

daily_sales_lower_bound = daily_sales_bounds['lower_bound'].values[0]
daily_sales_upper_bound = daily_sales_bounds['upper_bound'].values[0]

# 定义约束函数
def constraints(X):
    # 初始化约束列表
    con = []
    
    # 42个模型约束
    x = X[:42]
    y = X[42:]
    con.extend(y - a_values * np.exp(b_values * x))
    
    # 12个成本加价范围约束
    for i in range(6):
        con.append(markup_lower_bound[i] - np.min(x[i*7:i*7+7]))
        con.append(np.max(x[i*7:i*7+7]) - markup_upper_bound[i])
        
    # 12个销量范围约束
    for i in range(6):
        con.append(sales_lower_bound[i] - np.min(y[i*7:i*7+7]))
        con.append(np.max(y[i*7:i*7+7]) - sales_upper_bound[i])
        
    # 每日销售总量的上下界约束
    con.append(daily_sales_upper_bound - np.sum(y))
    con.append(np.sum(y) - daily_sales_lower_bound)
    
    return con

# 参数范围：x和y都在0-100之间（可根据实际情况调整）
lb = [0]*84
ub = [200]*84

# 使用PSO算法进行优化
xopt, fopt = pso(objective, lb, ub, f_ieqcons=constraints)

xopt, fopt
 


Stopping search: maximum iterations reached --> 100
However, the optimization couldn't find a feasible design. Sorry


(array([ 48.40600179, 124.05933439,  46.72941072,  33.23376332,
          2.16162518, 174.90979811,  91.4769874 , 117.75085243,
        186.90790583,  27.11098536,  86.50609337, 112.26837569,
        179.66080541, 111.384573  , 192.45057335,  22.57857448,
          1.16979461, 103.6208476 , 103.49490914,  94.42614548,
         89.21969449,  40.7422452 , 173.9483464 , 159.0163475 ,
         65.55728177,  86.37463009,  10.04624924,  47.9117054 ,
         45.59415381,  50.54196553, 189.3042607 , 107.67891822,
        156.67006938, 124.97269898, 178.92704627,  28.61885366,
          0.2209959 ,  56.04755793, 123.15074278, 141.53649882,
         62.88724919,  89.35952213,  50.24867675,  83.50142605,
        192.65679095,  16.58008315, 161.56324128,  47.39020088,
        127.47856955, 103.83815321,  22.88108101, 107.81304973,
         62.44699721,  25.4465675 , 102.2286415 , 104.70366422,
        100.47441031, 186.25288691,  95.59916728,   5.37746779,
          9.40420861,  58.42724205,  71.